# Neural Network for Spiral Data Classification

Implementation of a neural network to classify spiral data points into three classes (red, green, blue) using TensorFlow/Keras.

## 1. Import Required Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ModuleNotFoundError: No module named 'tensorflow'

## 2. Generate Spiral Data

In [ ]:
# Generate spiral data
N = 333  # number of points per class
K = 3    # number of classes
X = np.zeros((N*K, 2))  # data matrix (each row = single example)
y = np.zeros(N*K, dtype='uint8')  # class labels

for j in range(K):
    ix = range(N*j, N*(j+1))
    r = np.linspace(0.0, 1, N)  # radius
    t = np.linspace(j*4, (j+1)*4, N) + np.random.randn(N)*0.2  # theta
    X[ix] = np.c_[r*np.sin(t*2.5), r*np.cos(t*2.5)]
    y[ix] = j

# Plot the generated data
plt.figure(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu)
plt.title("Generated Spiral Data")
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()

## 3. Train-Test Split and Data Preprocessing

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert labels to one-hot encoding
y_train_onehot = np.eye(K)[y_train]
y_test_onehot = np.eye(K)[y_test]

# Visualize scaled training data
plt.figure(figsize=(8, 8))
plt.scatter(X_train_scaled[:, 0], X_train_scaled[:, 1], c=y_train, cmap=plt.cm.RdYlBu)
plt.title("Scaled Training Data")
plt.xlabel("X1 (scaled)")
plt.ylabel("X2 (scaled)")
plt.show()

## 4. Define Neural Network Architecture

In [ ]:
# Create the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 5. Model Training

In [ ]:
# Train the model
history = model.fit(X_train_scaled, y_train_onehot,
                   epochs=50,
                   batch_size=32,
                   validation_data=(X_test_scaled, y_test_onehot))

## 6. Model Evaluation and Visualization

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

# Make predictions on test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

# Plot true vs predicted classes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.RdYlBu)
ax1.set_title('True Classes')
ax1.set_xlabel('X1')
ax1.set_ylabel('X2')

ax2.scatter(X_test[:, 0], X_test[:, 1], c=y_pred_classes, cmap=plt.cm.RdYlBu)
ax2.set_title('Predicted Classes')
ax2.set_xlabel('X1')
ax2.set_ylabel('X2')

plt.tight_layout()
plt.show()

## 7. Test with New Points

In [ ]:
# Test a new point
new_point = np.array([[0.1, -5]])
new_point_scaled = scaler.transform(new_point)
prediction = model.predict(new_point_scaled)
predicted_class = np.argmax(prediction)

# Plot the result
plt.figure(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, alpha=0.5)
plt.scatter(new_point[0, 0], new_point[0, 1], c='yellow', s=200, label='New Point')
plt.title(f'New Point Classification\nPredicted Class: {predicted_class}')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()
plt.show()

## 8. Compare Different Optimizers

In [ ]:
# Function to create and train model with different optimizers
def train_with_optimizer(optimizer_name):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(2,)),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    
    if optimizer_name == 'sgd':
        optimizer = 'sgd'
    elif optimizer_name == 'sgd_momentum':
        optimizer = tf.keras.optimizers.SGD(momentum=0.9)
    else:
        optimizer = 'adam'
        
    model.compile(optimizer=optimizer,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    history = model.fit(X_train_scaled, y_train_onehot,
                       epochs=50,
                       batch_size=32,
                       validation_data=(X_test_scaled, y_test_onehot),
                       verbose=0)
    return history

# Train with different optimizers
optimizers = ['adam', 'sgd', 'sgd_momentum']
histories = {opt: train_with_optimizer(opt) for opt in optimizers}

# Plot comparison
plt.figure(figsize=(12, 4))

# Plot training accuracy
plt.subplot(1, 2, 1)
for opt in optimizers:
    plt.plot(histories[opt].history['accuracy'], label=opt)
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot validation accuracy
plt.subplot(1, 2, 2)
for opt in optimizers:
    plt.plot(histories[opt].history['val_accuracy'], label=opt)
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()